In [1]:
from math import sqrt
import torch
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from PIL import Image
from xml.dom.minidom import parse
import h5py
import numpy as np
import torchvision
import random
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import models
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import cv2
from engine import train_one_epoch, evaluate
import utils
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm import tqdm
import warnings
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.transforms import functional as F
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN
from pycocotools.mask import encode as encode


In [2]:
img_list = os.listdir(r'dataset\test')
img_id = [2, 1, 4, 3, 5, 6]


In [3]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


In [4]:

model = torch.load('model.pt')
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)


In [5]:
answer = []

for i in range(6):
    for x in range(4):
        for y in range(4):
            img = Image.open('dataset/test/' + img_list[i])
            xmin, ymin, xmax, ymax = 0+x*250, 0+y*250, 250+x*250, 250+y*250
            # print(xmin,ymin,xmax,ymax)
            img = img.crop((xmin, ymin, xmax, ymax))
            img = transform(img)
            model.eval()
            with torch.no_grad():
                prediction = model([img.to(device)])
            for ii in range(min(prediction[0]['boxes'].cpu().shape[0], 2000)):

                box_xmin = round(prediction[0]['boxes'][ii][0].item())
                box_ymin = round(prediction[0]['boxes'][ii][1].item())
                box_xmax = round(prediction[0]['boxes'][ii][2].item())
                box_ymax = round(prediction[0]['boxes'][ii][3].item())

                if (box_xmin == 0 and x != 0) or (box_xmax >= 249 and x != 3):
                    continue
                if (box_ymin == 0 and y != 0) or (box_ymax >= 249 and y != 3):
                    continue

                box_x, box_y, box_width, box_height = \
                    box_xmin, box_ymin, box_xmax - \
                    box_xmin, box_ymax-box_ymin

                mask = prediction[0]['masks'][ii][0].cpu()
                mask = np.array(mask)
                mask = cv2.copyMakeBorder(
                    mask, ymin-0, 1000-ymax, xmin-0,
                    1000-xmax, cv2.BORDER_CONSTANT, value=(0, 0, 0))
                mask[np.where(mask > 0.7)] = 1
                # print(mask.shape)
                mask = np.asfortranarray(mask).astype(np.uint8)
                mask_counts = (encode(np.asfortranarray(mask))
                               ['counts']).decode('utf8')
                # print(sqrt((xmax - xmin)*(ymax-ymin)))
                label = prediction[0]['labels'][ii].item()
                score = prediction[0]['scores'][ii].item()

                dict = {}
                dict['image_id'] = img_id[i]
                dict['bbox'] = [box_x+xmin, box_y+ymin, box_width, box_height]
                dict['score'] = float(score)
                dict['category_id'] = 1
                dict['segmentation'] = {
                    "size": [1000, 1000], 'counts': mask_counts}
                answer.append(dict)

    for x in range(3):
        for y in range(4):
            img = Image.open('dataset/test/' + img_list[i])
            xmin, ymin, xmax, ymax = 125+x*250, 0+y*250, 375+x*250, 250+y*250
            # print(xmin,ymin,xmax,ymax)
            img = img.crop((xmin, ymin, xmax, ymax))
            img = transform(img)
            model.eval()
            with torch.no_grad():
                prediction = model([img.to(device)])
            for ii in range(min(prediction[0]['boxes'].cpu().shape[0], 2000)):

                box_xmin = round(prediction[0]['boxes'][ii][0].item())
                box_ymin = round(prediction[0]['boxes'][ii][1].item())
                box_xmax = round(prediction[0]['boxes'][ii][2].item())
                box_ymax = round(prediction[0]['boxes'][ii][3].item())
                if (box_ymin == 0 and y != 0) or (box_ymax >= 249 and y != 3):
                    continue
                if(box_xmin < 125 and box_xmax > 125):
                    pass
                else:
                    continue

                box_x, box_y, box_width, box_height = \
                    box_xmin, box_ymin, box_xmax - \
                    box_xmin, box_ymax-box_ymin

                mask = prediction[0]['masks'][ii][0].cpu()
                mask = np.array(mask)
                mask = cv2.copyMakeBorder(
                    mask, ymin-0, 1000-ymax, xmin-0, 1000-xmax,
                    cv2.BORDER_CONSTANT, value=(0, 0, 0))
                mask[np.where(mask > 0.7)] = 1
                mask = np.asfortranarray(mask).astype(np.uint8)
                mask_counts = (encode(np.asfortranarray(mask))
                               ['counts']).decode('utf8')
                # print(sqrt((xmax - xmin)*(ymax-ymin)))
                label = prediction[0]['labels'][ii].item()
                score = prediction[0]['scores'][ii].item()

                dict = {}
                dict['image_id'] = img_id[i]
                dict['bbox'] = [box_x+xmin, box_y+ymin, box_width, box_height]
                dict['score'] = float(score)
                dict['category_id'] = 1
                dict['segmentation'] = {
                    "size": [1000, 1000], 'counts': mask_counts}
                answer.append(dict)

    for x in range(4):
        for y in range(3):
            img = Image.open('dataset/test/' + img_list[i])
            xmin, ymin, xmax, ymax = 0+x*250, 125+y*250, 250+x*250, 375+y*250
            # print(xmin,ymin,xmax,ymax)
            img = img.crop((xmin, ymin, xmax, ymax))
            img = transform(img)
            model.eval()
            with torch.no_grad():
                prediction = model([img.to(device)])
            for ii in range(min(prediction[0]['boxes'].cpu().shape[0], 2000)):

                box_xmin = round(prediction[0]['boxes'][ii][0].item())
                box_ymin = round(prediction[0]['boxes'][ii][1].item())
                box_xmax = round(prediction[0]['boxes'][ii][2].item())
                box_ymax = round(prediction[0]['boxes'][ii][3].item())
                if (box_xmin == 0 and x != 0) or (box_xmax >= 249 and x != 3):
                    continue
                if(box_ymin < 125 and box_ymax > 125):
                    pass
                else:
                    continue

                box_x, box_y, box_width, box_height = \
                    box_xmin, box_ymin, box_xmax - \
                    box_xmin, box_ymax-box_ymin

                mask = prediction[0]['masks'][ii][0].cpu()
                mask = np.array(mask)
                mask = cv2.copyMakeBorder(
                    mask, ymin-0, 1000-ymax,
                    xmin-0, 1000-xmax, cv2.BORDER_CONSTANT, value=(0, 0, 0))
                mask[np.where(mask > 0.7)] = 1
                mask = np.asfortranarray(mask).astype(np.uint8)
                mask_counts = (encode(np.asfortranarray(mask))
                               ['counts']).decode('utf8')
                # print(sqrt((xmax - xmin)*(ymax-ymin)))
                label = prediction[0]['labels'][ii].item()
                score = prediction[0]['scores'][ii].item()

                dict = {}
                dict['image_id'] = img_id[i]
                dict['bbox'] = [box_x+xmin, box_y+ymin, box_width, box_height]
                dict['score'] = float(score)
                dict['category_id'] = 1
                dict['segmentation'] = {
                    "size": [1000, 1000], 'counts': mask_counts}
                answer.append(dict)

    for x in range(3):
        for y in range(3):
            img = Image.open('dataset/test/' + img_list[i])
            xmin, ymin, xmax, ymax = 125+x*250, 125+y*250, 375+x*250, 375+y*250
            img = img.crop((xmin, ymin, xmax, ymax))
            img = transform(img)
            model.eval()
            with torch.no_grad():
                prediction = model([img.to(device)])
            for ii in range(min(prediction[0]['boxes'].cpu().shape[0], 2000)):

                box_xmin = round(prediction[0]['boxes'][ii][0].item())
                box_ymin = round(prediction[0]['boxes'][ii][1].item())
                box_xmax = round(prediction[0]['boxes'][ii][2].item())
                box_ymax = round(prediction[0]['boxes'][ii][3].item())
                if(box_xmin < 125 and box_xmax > 125):
                    pass
                else:
                    continue
                if(box_ymin < 125 and box_ymax > 125):
                    pass
                else:
                    continue

                box_x, box_y, box_width, box_height = \
                    box_xmin, box_ymin, box_xmax - \
                    box_xmin, box_ymax-box_ymin

                mask = prediction[0]['masks'][ii][0].cpu()
                mask = np.array(mask)
                mask = cv2.copyMakeBorder(
                    mask, ymin-0, 1000-ymax, xmin-0, 1000-xmax,
                    cv2.BORDER_CONSTANT, value=(0, 0, 0))
                mask[np.where(mask > 0.7)] = 1
                mask = np.asfortranarray(mask).astype(np.uint8)
                mask_counts = (encode(np.asfortranarray(mask))
                               ['counts']).decode('utf8')
                label = prediction[0]['labels'][ii].item()
                score = prediction[0]['scores'][ii].item()

                dict = {}
                dict['image_id'] = img_id[i]
                dict['bbox'] = [box_x+xmin, box_y+ymin, box_width, box_height]
                dict['score'] = float(score)
                dict['category_id'] = 1
                dict['segmentation'] = {
                    "size": [1000, 1000], 'counts': mask_counts}
                answer.append(dict)


C:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
import json
jsObj = json.dumps(answer, indent=4)
with open('answer.json', "w") as f:
    f.write(jsObj)
    f.close()
